In [62]:
import os 
import numpy as np
import pandas as pd

In [63]:
folder_path = 'Jesse_Dataset_v2/Pure_Raman_txt'
output_csv = 'reference_v2.csv'

reference_file = '1,9-nonanedithiol - 1.txt'
min_wn, max_wn = 176, 2445

# Step 1: Get reference axis
ref_path = os.path.join(folder_path, reference_file)
df_ref = pd.read_csv(ref_path, delim_whitespace=True, header=None)
wavenumber_axis = df_ref.iloc[:, 0].values
mask = (wavenumber_axis > min_wn) & (wavenumber_axis < max_wn)
cropped_axis = np.round(wavenumber_axis[mask], 2)

# Step 2: Process all files
txt_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.txt')])
data = []
labels = []


C:\Users\levinej\AppData\Local\Temp\ipykernel_15516\3142951186.py:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_ref = pd.read_csv(ref_path, delim_whitespace=True, header=None)


In [64]:
for file in txt_files:
    path = os.path.join(folder_path, file)
    try:
        df = pd.read_csv(path, delim_whitespace=True, header=None)
        wn = df.iloc[:, 0].values
        intensity = df.iloc[:, 1].values

        # Crop to same wavenumber window
        mask = (wn > min_wn) & (wn < max_wn)
        wn_cropped = wn[mask]
        intensity_cropped = intensity[mask]

        # Align: make sure axis matches reference length
        if len(wn_cropped) != len(cropped_axis):
            raise ValueError(f"Wavenumber mismatch in {file} ({len(wn_cropped)} vs {len(cropped_axis)})")

        data.append(intensity_cropped)
        label = os.path.splitext(file)[0].split('_')[0]
        labels.append(label)

    except Exception as e:
        print(f"⚠️ Error with file {file}: {e}")
        data.append(np.full_like(cropped_axis, np.nan))
        labels.append("Error")

C:\Users\levinej\AppData\Local\Temp\ipykernel_15516\1243765178.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(path, delim_whitespace=True, header=None)
C:\Users\levinej\AppData\Local\Temp\ipykernel_15516\1243765178.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(path, delim_whitespace=True, header=None)
C:\Users\levinej\AppData\Local\Temp\ipykernel_15516\1243765178.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(path, delim_whitespace=True, header=None)
C:\Users\levinej\AppData\Local\Temp\ipykernel_15516\1243765178.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version.

In [65]:
df_out = pd.DataFrame(data, columns=cropped_axis)
df_out['Label'] = labels
df_out.to_csv(output_csv, index=False)

print(f"✅ Saved {len(data)} spectra to '{output_csv}' using axis from '{reference_file}' cropped to ({min_wn}, {max_wn}).")


✅ Saved 565 spectra to 'reference_v2.csv' using axis from '1,9-nonanedithiol - 1.txt' cropped to (176, 2445).
